In [ ]:
import theano.tensor as T
from theano import shared, config, function
import theano
import numpy as np
# import theano.tensor.sharedvar.ScalarSharedVariable
# import theano.tensor.sharedvar.TensorSharedVariable

# test replacing shared variables values

In [ ]:
s1 = shared(np.array(1, dtype=config.floatX))  # note that non-numpy value will result in a different sharedvariabletype which then afterwards cannot be changed back
s2 = shared(np.array([1,3,1], dtype=config.floatX))

solution to this problem is to define a custom shared function which wrapps everything into a numpy value (or gnumpy possibly, the main goal is to get TensorSharedVariables by default)

    def myshared(v):
        return shared(np.array(v, dtype=config.floatX))

In [ ]:
parameters = [s1, s2]

In [ ]:
vs = []
shapes = []
sizes = []
for p in parameters:
    v = p.get_value(borrow=True)
    shape = v.shape
    size = np.prod(shape)
    
    vs.append(v)
    shapes.append(shape)
    sizes.append(int(size))

In [ ]:
alltogether = np.empty(int(sum(sizes)), dtype=config.floatX)
i = 0
for p, v, shape, size in zip(parameters, vs, shapes, sizes):
    alltogether[i:i+size] = v.flat
    p.set_value(alltogether[i:i+size].reshape(shape), borrow=True)
    i += size

In [ ]:
alltogether

In [ ]:
alltogether[:] = 6
alltogether

In [ ]:
s2.get_value()

In [ ]:
s1.get_value()

it works to replace all shared values by a common alltogether reference

# scipy test

In [ ]:
from scipy.optimize import minimize

In [ ]:
global_xs = np.arange(10)

In [ ]:
def f(xs):
    return abs(xs).sum()

In [ ]:
minimize(f, global_xs, options={'maxiter':50})

In [ ]:
global_xs

scipy doesn't work in place unfortunately

# test symbolic shared variables

In [22]:
import theano.tensor as T
from theano import shared, config, function
import theano
import numpy as np
# import theano.tensor.sharedvar.ScalarSharedVariable
# import theano.tensor.sharedvar.TensorSharedVariable

In [23]:
from theano_helpers import shared

In [24]:
v = T.dvector()
v.dtype

'float64'

In [25]:
sv = shared(T.ones((v.size,)))

In [26]:
sv.get_value()

array([ 0.])

In [27]:
sv.eval({v:np.array([1,2,3])})

array([ 1.,  1.,  1.])

In [28]:
sv.eval()

array([ 1.,  1.,  1.])

In [29]:
sv2 = sv * 2

In [30]:
sv2.eval()

array([ 2.,  2.,  2.])

# test scalartensor shared

``from theano_helpers import shared`` defines also a custom tensor_constructur which makes everything into a TensorSharedVariable such that we can ensure changebility afterwards.

It furthermore defaults to using floatX

In [3]:
s1 = shared(1)

In [4]:
s2 = shared([[1,3,1]])

In [7]:
sss = T.join(0, s2, s2)

In [8]:
sss.eval()

array([[ 1.,  3.,  1.],
       [ 1.,  3.,  1.]])

In [9]:
parameters = [s1, s2]

In [10]:
vs = []
shapes = []
sizes = []
for p in parameters:
    v = p.get_value(borrow=True)
    shape = v.shape
    size = np.prod(shape)
    
    vs.append(v)
    shapes.append(shape)
    sizes.append(int(size))

In [11]:
alltogether = np.empty(int(sum(sizes)), dtype=config.floatX)
i = 0
for p, v, shape, size in zip(parameters, vs, shapes, sizes):
    alltogether[i:i+size] = v.flat
    p.set_value(alltogether[i:i+size].reshape(shape), borrow=True)
    i += size

In [12]:
alltogether

array([ 1.,  1.,  3.,  1.])

In [13]:
alltogether[:] = 6
alltogether

array([ 6.,  6.,  6.,  6.])

In [14]:
s2.get_value()

array([[ 6.,  6.,  6.]])

In [15]:
s1.get_value()

array(6.0)

works now by default

# test proxify on shape values

In [ ]:
from schlichtanders.mymeta import proxify

In [ ]:
a = T.dscalar()

In [ ]:
v = T.dvector()

In [ ]:
v2 = T.dvector()

In [ ]:
T.ones(a)

In [ ]:
s = shared(T.ones((v.size)))

In [ ]:
s1 = shared([1,2,3])
s2 = shared([1,2,3,4,5])

In [ ]:
shape1 = s1.shape

In [ ]:
proxify(s1, s2)

In [ ]:
shape1.eval()